In [177]:
print("Ok")

Ok


In [178]:
from utils.model_loader import ModelLoader

In [179]:
model = ModelLoader()

{"timestamp": "2025-10-24T12:33:35.113960Z", "level": "warning", "event": "OPENAI_API_KEY is missing from environment"}
{"timestamp": "2025-10-24T12:33:35.114634Z", "level": "info", "event": "GOOGLE_API_KEY loaded from environment"}
{"timestamp": "2025-10-24T12:33:35.115098Z", "level": "info", "event": "GROQ_API_KEY loaded from environment"}
{"timestamp": "2025-10-24T12:33:35.115429Z", "level": "warning", "event": "ASTRA_DB_API_ENDPOINT is missing from environment"}
{"timestamp": "2025-10-24T12:33:35.115824Z", "level": "warning", "event": "ASTRA_DB_APPLICATION_TOKEN is missing from environment"}
{"timestamp": "2025-10-24T12:33:35.116187Z", "level": "warning", "event": "ASTRA_DB_KEYSPACE is missing from environment"}
{"config_keys": ["astra_db", "embedding_model", "retriever", "llm"], "timestamp": "2025-10-24T12:33:35.118235Z", "level": "info", "event": "YAML config loaded"}


In [180]:
llm = model.load_llm()

{"provider": "groq", "model": "openai/gpt-oss-120b", "timestamp": "2025-10-24T12:33:35.312609Z", "level": "info", "event": "Loading LLM"}


In [181]:
llm.invoke("Hi").content

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


'Hello! How can I help you today?'

In [182]:
from typing import List
from typing_extensions import TypedDict
from pydantic import BaseModel, Field

In [183]:
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display

In [184]:
class Analyst(BaseModel):
    name: str = Field(description="Name of the analyst.")
    role: str = Field(description="role of the analyst.")
    affiliation: str = Field(description="Primary Affiliation of the analyst")
    description: str = Field(description="Description of the analyst focus, concerns and motives.")

    @property
    def persona(self) -> str:
        return f"Name: {self.name}\nRole: {self.role}\nAffiliation: {self.affiliation}\nDescription: {self.description}"

In [185]:
Analyst( name="Dr. Neha Patel", role="Medical Data Scientist", affiliation="Stanford Medicine", description="Focuses on predictive models for patient outcomes." )
Analyst( name="Dr. Arun Verma", role="Ethics Researcher", affiliation="WHO", description="Explores ethical implications of AI in diagnostics." )
Analyst( name="Ms. Priya Sharma", role="Policy Analyst", affiliation="Ministry of Health", description="Investigates AI policy and compliance frameworks." )

Analyst(name='Ms. Priya Sharma', role='Policy Analyst', affiliation='Ministry of Health', description='Investigates AI policy and compliance frameworks.')

In [186]:
class Perspective(BaseModel):
    analysts: List[Analyst] = Field(description="Comprehensive list of analysts with their roles and affiliations")

In [ ]:
class GenerateAnalystsState(TypedDict):
    topic: str # Research Topics
    max_analyst: int # number of analyst
    human_analyst_feedback: str # Human Feedback
    analysts: List[Analyst] # Analyst asking questions


In [ ]:
GenerateAnalystsState(
    topic="Finance",
    max_analyst=5,
    human_analyst_feedback="Give me real info"
)

{'topic': 'Finance',
 'max_analysts': 5,
 'human_analyst_feedback': 'Give me real info'}

In [189]:
analyst_instructions="""You are tasked with creating a set of AI analyst personas. Follow these instructions carefully:

1. First, review the research topic:
{topic}
        
2. Examine any editorial feedback that has been optionally provided to guide creation of the analysts: 
        
{human_analyst_feedback}
    
3. Determine the most interesting themes based upon documents and / or feedback above.
                    
4. Pick the top {max_analysts} themes.

5. Assign one analyst to each theme."""

In [ ]:
print([analyst_instructions.format(
        topic="education",
        max_analyst=4,
        human_analyst_feedback="please exaplain only on AI"
    )]+["Generate the set of analysts."])

['You are tasked with creating a set of AI analyst personas. Follow these instructions carefully:\n\n1. First, review the research topic:\neducation\n\n2. Examine any editorial feedback that has been optionally provided to guide creation of the analysts: \n\nplease exaplain only on AI\n\n3. Determine the most interesting themes based upon documents and / or feedback above.\n\n4. Pick the top 4 themes.\n\n5. Assign one analyst to each theme.', 'Generate the set of analysts.']


In [191]:
def create_analyst(state: GenerateAnalystsState):
    """
        It is creating my analyst
    """

    topic = state['topic']
    max_analyst = state['max_analyst']
    human_analyst_feedback = state.get('human_analyst_feedback', "")

    structured_llm = llm.with_structured_output(Perspective)
    system_message = analyst_instructions.format(
        topic=topic,
        max_analysts=max_analyst,
        human_analyst_feedback= human_analyst_feedback
    )

    analysts = structured_llm.invoke([SystemMessage(content=system_message)] + [HumanMessage(content="Generate the set of analysts.")])
    return {"analysts": analysts}

In [194]:
print(create_analyst({
    'topic': "health",
    'max_analyst': 5,
    'human_analyst_feedback': "Get me real info"
}))

HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


{'analysts': Perspective(analysts=[Analyst(name='Dr. Maya Patel', role='Epidemiology Analyst', affiliation='World Health Organization (WHO)', description='Focuses on global disease surveillance, outbreak response, and population health trends. Prioritizes evidence-based policy recommendations and real-time data accuracy.'), Analyst(name='Dr. Luis Ramirez', role='Mental Health Researcher', affiliation='National Institute of Mental Health (NIMH)', description='Specializes in community mental health, stigma reduction, and the impact of socioeconomic factors on psychological well‑being. Emphasizes peer‑reviewed studies and longitudinal data.'), Analyst(name='Prof. Aisha Khan', role='Nutrition & Lifestyle Analyst', affiliation='Harvard T.H. Chan School of Public Health', description='Examines dietary patterns, food security, and the relationship between nutrition and chronic disease. Relies on large cohort studies, meta‑analyses, and real‑world dietary surveys.'), Analyst(name='Dr. Ethan Li

In [115]:
def human_feedback(state):
    """
        Args: 
            state (_type_): description
    """
    pass

In [116]:
def should_continue(state):
    """
    Args:
        state (_type_): description
    """
    pass 

In [118]:
builder = StateGraph()

TypeError: StateGraph.__init__() missing 1 required positional argument: 'state_schema'

In [100]:
builder.add_node()
builder.add_node()

NameError: name 'builder' is not defined

In [101]:
builder.edge(START, create_analyst)
builder.add_edge("create_analyst", "human_feedback")
builder.conditional_edge("human_feedback", should_continue, ["create_analyst", END])

NameError: name 'builder' is not defined

In [102]:
memory = MemorySaver()

In [103]:
graph = builder.compile(interrupt_before=["human_feedback"], checkpointer=memory)

NameError: name 'builder' is not defined

In [104]:
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

NameError: name 'graph' is not defined